# Chapter 10. 데이터 집계와 그룹 연산

- pandas는 데이터 집합을 자연스럽게 나누고 요약할 수 있는 groupby라는 유연한 방법을 제공함.
- 파이썬과 pandas의 강력한 표현력을 잘 이용하면 아주 복잡한 그룹 연산도 pandas 객체나 NumPy 배열을 받는 함수의 조합으로 해결할 수 있음.
- 해당 Chapter에서는 다음과 같은 내용을 배우게 된다.
    - 하나 이상의 키(함수, 배열, DataFrame의 컬럼 이름)를 이용해서 pandas 객체를 여러 조각으로 나누는 방법
    - 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
    - 정규화, 선형회귀, 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
    - 피벗테이블과 교차일람표를 구하는 방법
    - 변위치 분석과 다른 통게 집단 분석을 수행하는 방법

## 10.1 GroupBy 메카닉

- 그룹 연산은 `분리-적용-결합`으로 이해하면 편하다.
- 각 그룹의 색인은 다음과 같이 다양한 형태가 될 수 있으며, 모두 같은 타입일 필요도 없다.
    - 그룹을 묶을 축과 동일한 길이의 리스트나 배열
    - DataFrame의 컬럼 이름을 지칭하는 값
    - 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
    - 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수
- 위 목록에서 마지막 세 방법은 객체를 나눌 때 사용할 배열을 생성하기 위한 방법임을 기억하자.

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame({'key1' : ['a','a','b','b','a'],
                   'key2' : ['one','two','one','two','one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.306061,1.212510
1,a,two,-0.764874,-0.574943
2,b,one,0.733001,-1.084389
3,b,two,0.020519,0.906184
4,a,one,1.590487,0.400596


> 이 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구해보자.   
> 여러가지 방법이 있지만 그 중 하나는 data1에 대해 groupby 메서드를 호출하고 key1 컬럼을 넘기는 것이다.

In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [6]:
grouped.mean()

key1
a    0.377225
b    0.376760
Name: data1, dtype: float64

> 만약 여러 개의 배열을 리스트로 넘겼다면 조금 다른 결과를 얻을 수 있다.

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.948274
      two    -0.764874
b     one     0.733001
      two     0.020519
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,two
key1,,
a,0.948274,-0.764874
b,0.733001,0.020519


> 길이만 같다면 그룹의 색인은 어떤 배열이라도 상관없다.

In [12]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.764874
            2006    0.733001
Ohio        2005    0.163290
            2006    1.590487
Name: data1, dtype: float64

> 한 그룹으로 묶을 정보는 주로 같은 DataFrame 안에서 찾게 되는데, 이 경우 컬럼 이름(문자열, 숫자 혹은 다른 파이썬 객체)을 넘겨서 그룹의 색인으로 사용할 수 있음.  

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.377225,0.346055
b,0.376760,-0.089103


> 위 결과를 보면 key2 컬럼이 결과에서 빠져 있는 것을 확인할 수 있다. 그 이유는 df['key2']는 숫자 데이터가 아니기 때문임.  
> 이런 컬럼은 성가신 컬럼 (nuisance column)이라고 부르며 결과에서 제외시킴.

In [16]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.948274  0.806553
     two  -0.764874 -0.574943
b    one   0.733001 -1.084389
     two   0.020519  0.906184

> groupby를 쓰는 목적과 별개로, 일반적으로 유용한 GroupBy 메서드는 `그룹의 크기`를 담고 있는 Series를 반환하는 Size 메서드다.

In [17]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 그룹 간 순회하기

> GroupBy 객체는 이터레이션을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환함.

In [22]:
for name, group in df.groupby('key1') : 
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.306061  1.212510
1    a  two -0.764874 -0.574943
4    a  one  1.590487  0.400596
b
  key1 key2     data1     data2
2    b  one  0.733001 -1.084389
3    b  two  0.020519  0.906184


> 색인이 여럿 존재하는 경우 튜플의 첫 번째 원소가 색인값이 됨.

In [23]:
for (k1,k2), group in df.groupby(['key1','key2']) :
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.306061  1.212510
4    a  one  1.590487  0.400596
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.764874 -0.574943
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.733001 -1.084389
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.020519  0.906184


> 한 줄이면 그룹별 데이터를 사전형으로 쉽게 바꿔서 유용하게 사용할 수 있음.

In [28]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.733001,-1.084389
3,b,two,0.020519,0.906184
